In [332]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import pyodbc
%matplotlib inline

Ticker = 'VWO'
model = '50-200 DC'
fileName = Ticker + '.csv'
df = pd.read_csv(fileName)

df.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3792 entries, 0 to 3791
Data columns (total 7 columns):
Date         3792 non-null object
Open         3792 non-null float64
High         3792 non-null float64
Low          3792 non-null float64
Close        3792 non-null float64
Adj Close    3792 non-null float64
Volume       3792 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 207.5+ KB


In [315]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2005-03-10,25.150000,25.250000,24.975000,25.080000,17.175423,108200
1,2005-03-11,25.190001,25.315001,25.045000,25.049999,17.154877,184400
2,2005-03-14,24.980000,25.055000,24.799999,24.900000,17.052153,112800
3,2005-03-15,24.625000,24.680000,24.350000,24.360001,16.682350,127800
4,2005-03-16,24.315001,24.385000,24.125000,24.150000,16.538532,196400


In [333]:
class Position:
    def __init__(self,entryPrice,entryDate,exitPrice,exitDate,profitLoss):
        self.entryPrice = entryPrice
        self.entryDate = entryDate
        self.exitPrice = exitPrice
        self.exitDate = exitDate
        self.profitLoss = profitLoss
    
def donchianChannelAI(nHigh,nLow,n,positions):
    posCount = len(positions)
    if(posCount == 0):
       #No position history exist
        if(n['Close'] >= nHigh):
            #Opening first position!
            newPosition = Position(n['Close'],n['Date'],0,'01/01/1999',0)
            return [1,newPosition]
        else:
            #Entry criteria not met
            return [0,None]
    else:
        #Positions exist, get latest position
        currentPosition = positions[posCount-1]
        
        #latest position is still open
        if(currentPosition.exitPrice ==0):  
            #Check if close is below recent low
            if(n['Close'] < nLow):
                #price closed below recent low, update current position
                currentPosition.exitPrice = n['Close']
                currentPosition.exitDate = n['Date']
                currentPosition.profitLoss = n['Close']/currentPosition.entryPrice
                
                #return closed position to update latest item in 'Positions'
                return [2,currentPosition]
            else:
                #price closed above recent low, return null action
                return [0,None]
        else:
            #Latest position has been closed, check if new position is needed
            if(n['Close'] >= nHigh):
                newPosition = Position(n['Close'],n['Date'],0,'01/01/1999',0)
                return [1,newPosition]
            else:
                return [0,None]
    
        
def create(positions,ticker,model):
    conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=DESKTOP-E4GJV84\SQLEXPRESS;"
    "Database=QuantAnalysis;"
    "Trusted_Connection=yes;"
    )

    cursor = conn.cursor()
    for p in positions:
        cursor.execute("Insert Into tblModelResults (entryDate,entryPrice,exitDate,exitPrice, plPercent,ticker,model) VALUES(?,?,?,?,?,?,?)",p.entryDate,p.entryPrice,p.exitDate,p.exitPrice,p.profitLoss,ticker,model)
        conn.commit()
    conn.close()


In [334]:
highs = []
lows = []
nHigh = 0
nLow = 0
upperChannel = 50
lowerChannel = 200
i = 0
positions = []


for j,n in df.iterrows():
    highs.append(n['High'])
    lows.append(n['Low'])
    i +=1
    
    if i >= upperChannel:
        highs.remove(highs[0])
    if i >= lowerChannel:
        lows.remove(lows[0])
        
    if i >=upperChannel:
        pos = donchianChannelAI(nHigh,nLow,n,positions)
        
        if(pos[0]==1):
            #new position to add
            positions.append(pos[1])
        elif(pos[0]==2):
            #drop most recent position and replace with updated version
            positions.pop()
            positions.append(pos[1])
    
    nHigh = max(highs)
    nLow = min(lows)


In [335]:
#labels = ['EntryDate','EntryPrice','ExitDate','ExitPrice','PLPercent','Ticker','Model']
posList = []
plList = []
for p in positions:
    posList.append([p.entryDate,p.entryPrice,p.exitDate,p.exitPrice,p.profitLoss,Ticker,model])
    plList.append(p.profitLoss)

print(np.mean(plList))

1.2442128909134043


In [336]:
create(positions,Ticker,model)